# Day 17: Conway Cubes

As your flight slowly drifts through the sky, the Elves at the Mythical Information Bureau at the North Pole contact you. They'd like some help debugging a malfunctioning experimental energy source aboard one of their super-secret imaging satellites.

The experimental energy source is based on cutting-edge technology: a set of Conway Cubes contained in a pocket dimension! When you hear it's having problems, you can't help but agree to take a look.

The pocket dimension contains an infinite 3-dimensional grid. At every integer 3-dimensional coordinate (x,y,z), there exists a single cube which is either active or inactive.

In the initial state of the pocket dimension, almost all cubes start inactive. The only exception to this is a small flat region of cubes (your puzzle input); the cubes in this region start in the specified active (#) or inactive (.) state.

The energy source then proceeds to boot up by executing six cycles.

Each cube only ever considers its neighbors: any of the 26 other cubes where any of their coordinates differ by at most 1. For example, given the cube at x=1,y=2,z=3, its neighbors include the cube at x=2,y=2,z=2, the cube at x=0,y=2,z=3, and so on.

During a cycle, all cubes simultaneously change their state according to the following rules:

    If a cube is active and exactly 2 or 3 of its neighbors are also active, the cube remains active. Otherwise, the cube becomes inactive.
    If a cube is inactive but exactly 3 of its neighbors are active, the cube becomes active. Otherwise, the cube remains inactive.

The engineers responsible for this experimental energy source would like you to simulate the pocket dimension and determine what the configuration of cubes should be at the end of the six-cycle boot process.

For example, consider the following initial state:

```text
.#.
..#
###
```

Even though the pocket dimension is 3-dimensional, this initial state represents a small 2-dimensional slice of it. (In particular, this initial state defines a 3x3x1 region of the 3-dimensional space.)

Simulating a few cycles from this initial state produces the following configurations, where the result of each cycle is shown layer-by-layer at each given z coordinate (and the frame of view follows the active cells in each cycle):

Before any cycles:

```text
z=0
.#.
..#
###
```

After 1 cycle:

```text
z=-1
#..
..#
.#.

z=0
#.#
.##
.#.

z=1
#..
..#
.#.
```

After 2 cycles:

```text
z=-2
.....
.....
..#..
.....
.....

z=-1
..#..
.#..#
....#
.#...
.....

z=0
##...
##...
#....
....#
.###.

z=1
..#..
.#..#
....#
.#...
.....

z=2
.....
.....
..#..
.....
.....
```

After 3 cycles:

```text
z=-2
.......
.......
..##...
..###..
.......
.......
.......

z=-1
..#....
...#...
#......
.....##
.#...#.
..#.#..
...#...

z=0
...#...
.......
#......
.......
.....##
.##.#..
...#...

z=1
..#....
...#...
#......
.....##
.#...#.
..#.#..
...#...

z=2
.......
.......
..##...
..###..
.......
.......
.......
```

After the full six-cycle boot process completes, 112 cubes are left in the active state.

Starting with your given initial configuration, simulate six cycles. How many cubes are left in the active state after the sixth cycle?

In [1]:
# Python imports
from pathlib import Path
from typing import Set, Tuple

We could load this data in as a 3D array with "active" and "inactive" states explicitly coded, but that would be inefficient - we'd have to keep growing the array, and most values are likely to be inactive. Instead, we keep a "sparse array" by only recording the co-ordinates of active cubes.

Python provides `set` datatypes which are fast to search, and operate on. To begin with, we load the co-ordinates of active cubes into a set of x,y,z-co-ordinates.

In [2]:
def load_data(fpath: str) -> Set[Tuple[int]]:
    """Load starting data as a sparse array of active cubes
    
    :param fpath:  path to puzzle input data
    """
    active = set()
    with Path(fpath).open("r") as ifh:
        z = 0
        for y, line in enumerate([_.strip() for _ in ifh.readlines()]):
            for x, char in enumerate([_ for _ in line]):
                if char == "#":
                    active.add((x, y, z))
    return active

In [3]:
load_data("day17_test.txt")

{(0, 2, 0), (1, 0, 0), (1, 2, 0), (2, 1, 0), (2, 2, 0)}

We only need to check the status of immediate neighbours (+/-1 in any direction) of active cubes, so we're going to want a function that returns the locations of such neighbours of any specificed cube location. We write this as `neighbours()`:

In [4]:
def neighbours(cube: Tuple[int]) -> Set[Tuple[int]]:
    """Return coordinates of neighbours of the passed cube co-ordinate
    
    :param cube:  co-ordinates of a cube
    """
    x, y, z = cube
    neighbours = set()
    for xdiff in range(-1, 2):
        for ydiff in range(-1, 2):
            for zdiff in range(-1, 2):
                neighbours.add((x + xdiff, y + ydiff, z + zdiff))
    neighbours.remove(cube)
    return(neighbours)

Now, for the update set we want to be fairly efficient. Before we start, we have a set of `active` cubes. We iterate over each active cube and check each neighbour `nbr` in turn.

For each neighbour `nbr`, we check if we've already updated it. If we have, we skip it. If we have not seen it (check membership of a set called `seen`), we obtain the locations of its neighbours. Then we find the intersection between this set of locations, and the set of `active` cubes. The length of this intersection gives us the value we need to know how to update the location of `nbr`. We add the location `nbr` to the set of `seen` locations, and if it should be active we add it to a set called `next_cycle`.

This systematically checks each neighbour of an initially active cube to see how to update the cube, and the space around it.

In [5]:
def update(active: Set[Tuple[int]]) -> Set[Tuple[int]]:
    """Return co-ordinates of active cubes in the next cycle
    
    :param active:  co-ordinates of activate cubes in current cycle
    """
    # Only co-ordinates +/- 1 from current active set need to be checked;
    # and we only check unseen cubes
    seen = set()
    next_cycle = set()
    for cube in active:
        nbrs = neighbours(cube)
        for nbr in [_ for _ in nbrs if _ not in seen]:
            seen.add(nbr)
            active_nbrs = neighbours(nbr).intersection(active)
            if len(active_nbrs) in (2, 3) and nbr in cubes:
                next_cycle.add(nbr)
            elif len(active_nbrs) == 3:
                next_cycle.add(nbr)
    return next_cycle

In [6]:
cubes = load_data("day17_test.txt")
for cycle in range(6):
    cubes = update(cubes)
len(cubes)

112

In [7]:
cubes = load_data("day17_data.txt")
for cycle in range(6):
    cubes = update(cubes)
len(cubes)

223

## Part Two

For some reason, your simulated results don't match what the experimental energy source engineers expected. Apparently, the pocket dimension actually has four spatial dimensions, not three.

The pocket dimension contains an infinite 4-dimensional grid. At every integer 4-dimensional coordinate (x,y,z,w), there exists a single cube (really, a hypercube) which is still either active or inactive.

Each cube only ever considers its neighbors: any of the 80 other cubes where any of their coordinates differ by at most 1. For example, given the cube at x=1,y=2,z=3,w=4, its neighbors include the cube at x=2,y=2,z=3,w=3, the cube at x=0,y=2,z=3,w=4, and so on.

The initial state of the pocket dimension still consists of a small flat region of cubes. Furthermore, the same rules for cycle updating still apply: during each cycle, consider the number of active neighbors of each cube.

For example, consider the same initial state as in the example above. Even though the pocket dimension is 4-dimensional, this initial state represents a small 2-dimensional slice of it. (In particular, this initial state defines a 3x3x1x1 region of the 4-dimensional space.)

Simulating a few cycles from this initial state produces the following configurations, where the result of each cycle is shown layer-by-layer at each given z and w coordinate:

Before any cycles:

```text
z=0, w=0
.#.
..#
###
```

After 1 cycle:

```text
z=-1, w=-1
#..
..#
.#.

z=0, w=-1
#..
..#
.#.

z=1, w=-1
#..
..#
.#.

z=-1, w=0
#..
..#
.#.

z=0, w=0
#.#
.##
.#.

z=1, w=0
#..
..#
.#.

z=-1, w=1
#..
..#
.#.

z=0, w=1
#..
..#
.#.

z=1, w=1
#..
..#
.#.
```

After 2 cycles:

```text
z=-2, w=-2
.....
.....
..#..
.....
.....

z=-1, w=-2
.....
.....
.....
.....
.....

z=0, w=-2
###..
##.##
#...#
.#..#
.###.

z=1, w=-2
.....
.....
.....
.....
.....

z=2, w=-2
.....
.....
..#..
.....
.....

z=-2, w=-1
.....
.....
.....
.....
.....

z=-1, w=-1
.....
.....
.....
.....
.....

z=0, w=-1
.....
.....
.....
.....
.....

z=1, w=-1
.....
.....
.....
.....
.....

z=2, w=-1
.....
.....
.....
.....
.....

z=-2, w=0
###..
##.##
#...#
.#..#
.###.

z=-1, w=0
.....
.....
.....
.....
.....

z=0, w=0
.....
.....
.....
.....
.....

z=1, w=0
.....
.....
.....
.....
.....

z=2, w=0
###..
##.##
#...#
.#..#
.###.

z=-2, w=1
.....
.....
.....
.....
.....

z=-1, w=1
.....
.....
.....
.....
.....

z=0, w=1
.....
.....
.....
.....
.....

z=1, w=1
.....
.....
.....
.....
.....

z=2, w=1
.....
.....
.....
.....
.....

z=-2, w=2
.....
.....
..#..
.....
.....

z=-1, w=2
.....
.....
.....
.....
.....

z=0, w=2
###..
##.##
#...#
.#..#
.###.

z=1, w=2
.....
.....
.....
.....
.....

z=2, w=2
.....
.....
..#..
.....
.....
```

After the full six-cycle boot process completes, 848 cubes are left in the active state.

Starting with your given initial configuration, simulate six cycles in a 4-dimensional space. How many cubes are left in the active state after the sixth cycle?

As we chose the sparse array-like representation, the update to the original functions is trivial: the addition of a fourth co-ordinate in `load_data_4d()`, and an additional inner loop for `neighbours()`:

In [8]:
def load_data_4d(fpath: str) -> Set[Tuple[int]]:
    """Load starting data as a sparse array of active cubes
    
    :param fpath:  path to puzzle input data
    """    
    active = set()
    with Path(fpath).open("r") as ifh:
        z, w = 0, 0
        for y, line in enumerate([_.strip() for _ in ifh.readlines()]):
            for x, char in enumerate([_ for _ in line]):
                if char == "#":
                    active.add((x, y, z, w))
    return active

In [9]:
def neighbours_4d(cube: Tuple[int]) -> Set[Tuple[int]]:
    """Return coordinates of neighbours of the passed cube co-ordinate
    
    :param cube:  co-ordinates of a cube
    """    
    x, y, z, w = cube
    neighbours = set()
    for xdiff in range(-1, 2):
        for ydiff in range(-1, 2):
            for zdiff in range(-1, 2):
                for wdiff in range(-1, 2):
                    neighbours.add((x + xdiff, y + ydiff, z + zdiff, w + wdiff))
    neighbours.remove(cube)
    return(neighbours)

In [10]:
def update_4d(active: Set[Tuple[int]]) -> Set[Tuple[int]]:
    """Return co-ordinates of active cubes in the next cycle
    
    :param active:  co-ordinates of activate cubes in current cycle
    """    
    # Only co-ordinates +/- 1 from current active set need to be checked;
    # and we only check unseen cubes
    seen = set()
    next_cycle = set()
    for cube in active:
        nbrs = neighbours_4d(cube)
        for nbr in [_ for _ in nbrs if _ not in seen]:
            seen.add(nbr)
            active_nbrs = neighbours_4d(nbr).intersection(active)
            if len(active_nbrs) in (2, 3) and nbr in cubes:
                next_cycle.add(nbr)
            elif len(active_nbrs) == 3:
                next_cycle.add(nbr)
    return next_cycle

In [11]:
cubes = load_data_4d("day17_test.txt")
for cycle in range(6):
    cubes = update_4d(cubes)
len(cubes)

848

In [12]:
cubes = load_data_4d("day17_data.txt")
for cycle in range(6):
    cubes = update_4d(cubes)
len(cubes)

1884